#CafChem tools for using a AutoDock Vina for docking ligands in proteins

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MauricioCafiero/CafChem/blob/main/notebooks/AutoDockVina_CafChem_nometal.ipynb)

## This notebook allows you to:
- Load a ligand via either a SMILES string or SDF file
- load a PDB
- choose protein chains and metal ions to keep, set a box center
- Dock with a set number of repetitions
- currently does not allow flexible residues

## Requirements:
- This notebook will a conda environment and load AutoDock Vina
- CPU runtime with high memory (to get more cores) is recommended.
- Currently uses an older runtime on colab, but we may be able to fix this by importing rdkit directly.

## Acknowledgements:
- [adapted from  this repo](https://github.com/kimjc95/computational-chemistry)
- You can cite their work [here.](https://zenodo.org/records/14881401)

## Set-up
- will need a restart

### conda environment preparation
- markdown The runtime will be restarted shortly.


In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:10
🔁 Restarting kernel...


### Install Dependencies
- (takes about 1 min 40 sec)


In [1]:
# @title
print('Installing dependencies... ', end='')
import subprocess

subprocess.run("git clone --depth=1 https://github.com/QVina/qvina.git", shell=True)
subprocess.run("chmod -R 755 /content/qvina", shell=True)
subprocess.run("mamba install -c conda-forge rdkit pdbfixer openbabel openmm mdanalysis prolif parmed nglview ipywidgets=7", shell=True)
subprocess.run("pip3 install meeko pdb4amber", shell=True)

from rdkit import RDLogger
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdFMCS
from rdkit.Chem.Lipinski import RotatableBondSmarts
from pdbfixer import PDBFixer
from openmm import *
from openmm.app import *
from openmm.unit import *
import os
import random
import locale
import warnings
import ipywidgets as widgets
from tqdm.notebook import tqdm_notebook as tqdm
from datetime import datetime
import zipfile
import nglview as nv
import MDAnalysis as mda
import prolif as plf
from MDAnalysis.analysis import distances
from google.colab import output, files
warnings.filterwarnings("ignore")
output.enable_custom_widget_manager()

!pip install gemmi

Installing dependencies... 

/usr/local/lib/python3.11/site-packages/MDAnalysis/topology/tables.py:52: DeprecationWarning: Deprecated in version 2.8.0
MDAnalysis.topology.tables has been moved to MDAnalysis.guesser.tables. This import point will be removed in MDAnalysis version 3.0.0
  warnings.warn(wmsg, category=DeprecationWarning)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 46.9 MB/s eta 0:00:00


In [ ]:
# @title
from google.colab import output
output.enable_custom_widget_manager()
print("widgets enableddone!")

In [4]:
import pandas as pd

# quick run

In [18]:
def quick_run(SMILES):
  SMILES = "CC=CC1=NC=C(N=C1)C"
  Ligand_number = "1"

  pH = 7.0
  FFtype = "GAFF"

  if FFtype == "MMFF94":
      charge_method = 'mmff94'
  else:
      charge_method = 'gasteiger'

  print("Optimizing the ligand structure...", end='')
  if SMILES == "":
      ligandfile = files.upload()
      ligandfile_name = next(iter(ligandfile))
      subprocess.run(f'obabel {ligandfile_name} -O ligand.sdf -p{pH} --ff {FFtype} --partialcharge {charge_method} --best --minimize --steps 100000 --sd', shell=True)
  else:
      subprocess.run(f'obabel -:"{SMILES}" -O ligand.sdf -p{pH} --gen3d --ff {FFtype} --partialcharge {charge_method} --best --minimize --steps 150000 --sd', shell=True)

  mol = Chem.rdmolfiles.SDMolSupplier("ligand.sdf")[0]
  Chem.rdmolops.SanitizeMol(mol)


  n_rot_bonds = Chem.rdMolDescriptors.CalcNumRotatableBonds(mol)
  n_amide_bonds = Chem.rdMolDescriptors.CalcNumAmideBonds(mol)

  select_fix = ''


  rot_atom_pairs = mol.GetSubstructMatches(RotatableBondSmarts)
  rot_bonds = list(set([mol.GetBondBetweenAtoms(*ap).GetIdx() for ap in rot_atom_pairs]))

  Chem.rdDepictor.Compute2DCoords(mol)


  def update_rotatable_bond(change):
      new_input = change.new
      if new_input.endswith(','):
          return
      if new_input == '':
          fix_bonds_idx = []
      else:
          fix_bonds_idx = [int(s) for s in new_input.split(',')]
      bond_colors = recolor_bonds(fix_bonds_idx)
      d.ClearDrawing()
      Chem.Draw.rdMolDraw2D.PrepareAndDrawMolecule(d, mol, legend='Rotable in red, Fixed in blue',
                                                  highlightBonds=rot_bonds, highlightBondColors=bond_colors)
      rot_bond_count = 0
      for bond in rot_bonds:
          if bond not in fix_bonds_idx:
              rot_bond_count += 1

  SMILES = Chem.rdmolfiles.MolToSmiles(mol)
  rotable = ""

  if select_fix == '':
      fix_bonds_idx = []
  else:
      fix_bonds_idx = [int(s) for s in select_fix.split(',')]

  for i in fix_bonds_idx:
      bond = mol.GetBondWithIdx(i)
      a1 = bond.GetBeginAtom().GetIdx()+1 # RDKit은 0부터 셈
      a2 = bond.GetEndAtom().GetIdx()+1
      rotable = rotable + ' -r "'+SMILES+'" -b '+str(a1)+" "+str(a2)

  subprocess.run(f'mk_prepare_ligand.py -i ligand.sdf -o ligand.pdbqt{rotable}', shell=True)

  if os.path.exists('ligand.pdbqt'):
      print('ligand PDBQT file has been generated.')
  else:
      print('Failed to generate ligand PDBQT file.')

  flexFlag = False
  blind_docking = False

  center_x = "0.53"
  center_y = "-0.044"
  center_z = "-26.454"
  len_x = "28"
  len_y = "28"
  len_z = "28"

  ligand_number = 1
  number_reps = 1

  best_scores = []
  for run_number in range(1,number_reps+1,1):

      output = f"ligand{Ligand_number}_out_{run_number}.pdbqt"

      exhaustiveness = 32


      if flexFlag:
          rigid = "receptor = receptor.pdbqt\n"
          flex_rec = "flex = receptor_flex.pdbqt\n"
      else:
          rigid = "receptor = receptor.pdbqt\n"
          flex_rec = ""

      with open('config', 'w') as f:
          f.write(rigid)
          f.write("ligand = ligand.pdbqt\n")
          f.write("center_x = "+str(center_x)+'\n')
          f.write("center_y = "+str(center_y)+'\n')
          f.write("center_z = "+str(center_z)+'\n')
          f.write("size_x = "+str(len_x)+'\n')
          f.write("size_y = "+str(len_y)+'\n')
          f.write("size_z = "+str(len_z)+'\n')
          f.write("num_modes = 10\n")
          f.write("out = "+output+'\n')
          f.write(f"log = docking_log{Ligand_number}_{run_number}.txt\n")
          f.write("exhaustiveness = "+str(exhaustiveness)+'\n')
          f.write(flex_rec)

      if blind_docking:
          p = subprocess.Popen('/content/qvina/bin/qvina-w --config config', shell=True, bufsize=0, stdout=subprocess.PIPE, encoding='utf-8')
      else:
          p = subprocess.Popen('/content/qvina/bin/qvina2.1 --config config', shell=True, bufsize=0, stdout=subprocess.PIPE, encoding='utf-8')

      while p.poll() == None:
          out = p.stdout.read(1)
          print(out, end='')

      docking_file = f"docking_log{Ligand_number}_{run_number}.txt"
      df = open(docking_file, 'r')
      lines = df.readlines()
      df.close()

      master_file = open("master_dock.txt", 'a')
      master_file.write(f"Molecule {ligand_number} ====================\n")
      for line in lines:
        master_file.write(line)
      master_file.close()

      for line in lines:
        parts = line.split()
        if len(parts) >1:
          if parts[0] == "1":
            best_scores.append(parts[1])
      #files.download(docking_file)
      #files.download(output)

      for score in best_scores:
        print(score)

      ligand_number += 1

In [16]:
df = pd.read_csv('close_mols_rosmarinic')
SMILES_list = df['SMILES'].to_list()

In [19]:
for smile in SMILES_list[:3]:
  quick_run(smile)

Optimizing the ligand structure...ligand PDBQT file has been generated.
############################################################################
# If you used Quick Vina 2 in your work, please cite:                      #
#                                                                          #
# Amr Alhossary, Stephanus Daniel Handoko, Yuguang Mu, and Chee-Keong Kwoh,#
# Fast, Accurate, and Reliable Molecular Docking with QuickVina 2,         #
# Bioinformatics (2015), doi: 10.1093/bioinformatics/btv082                #
#                                                                          #
# You are also encouraged to cite Quick Vina 1:                            #
# Stephanus Daniel Handoko, Xuchang Ouyang, Chinh Tran To Su, Chee Keong   #
# Kwoh, Yew Soon Ong,                                                      #
# QuickVina: Accelerating AutoDock Vina Using Gradient-Based Heuristics for#
# Global Optimization,                                                     #
# IE

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

-5.8


In [ ]:
want = [2,6,9]

for file in want:
  docking_file = f"ligand{file}_out_1.pdbqt"
  files.download(docking_file)

### Third party widgets toggle

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
# from google.colab import output
# output.disable_custom_widget_manager()

## Ligand Prep

In [22]:
#@markdown ### Prepare Ligand.pdbqt
#@markdown Enter the ligand's SMILES string.<br/>
#@markdown Leave it empty to upload your ligand file.

SMILES = "CC=CC1=NC=C(N=C1)C" #@param {type:"string"}
Ligand_number = "1" #@param {type:"string"}

#@markdown Set the reference pH to determine the protonation state of your ligand.
pH = 7 #@param {type:"slider", min:0.0, max:14.0, step:0.1}
#@markdown Select a forcefield to optimize the structure of your ligand.
FFtype = "GAFF" #@param ['UFF', 'GAFF', 'Ghemical', 'MMFF94', 'MMFF94s']

if FFtype == "MMFF94":
    charge_method = 'mmff94'
else:
    charge_method = 'gasteiger'

print("Optimizing the ligand structure...", end='')
if SMILES == "":
    ligandfile = files.upload()
    ligandfile_name = next(iter(ligandfile))
    subprocess.run(f'obabel {ligandfile_name} -O ligand.sdf -p{pH} --ff {FFtype} --partialcharge {charge_method} --best --minimize --steps 100000 --sd', shell=True)
else:
    subprocess.run(f'obabel -:"{SMILES}" -O ligand.sdf -p{pH} --gen3d --ff {FFtype} --partialcharge {charge_method} --best --minimize --steps 150000 --sd', shell=True)

#@markdown The final structure will be saved to 'ligand.sdf'.
print("done.")

#@markdown Please check your ligand's structure below. If the widget is empty, try with different file format.

view = nv.NGLWidget()
view.add_structure(nv.FileStructure('ligand.sdf'))
view._set_size('700px','500px')
display(view)

Optimizing the ligand structure...done.


NGLWidget()

### The rotable bonds will be shown in red.<br/>
 - Among the rotable bonds, type the indices of the bonds you want to rigidify.


In [23]:
# @markdown remove rotatable bonds
mol = Chem.rdmolfiles.SDMolSupplier("ligand.sdf")[0]
Chem.rdmolops.SanitizeMol(mol)

d = Chem.Draw.rdMolDraw2D.MolDraw2DCairo(700, 500)
d.drawOptions().addBondIndices = True
d.drawOptions().annotationFontScale = 0.85

n_rot_bonds = Chem.rdMolDescriptors.CalcNumRotatableBonds(mol)
n_amide_bonds = Chem.rdMolDescriptors.CalcNumAmideBonds(mol)

select_fix = widgets.Text(value='', placeholder='Enter the bonds you want to fix. ex) 11,13',
                          description='Index of the bonds to fix:', style={'description_width': 'initial'},
                          layout=widgets.Layout(width='700px'))

display_rotable = widgets.Text(value=str(n_rot_bonds), description='Number of rotable bonds :',
                               style={'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
display_amide = widgets.Text(value=str(n_amide_bonds), description='Number of amide bonds :',
                             style={'description_width': 'initial'}, layout=widgets.Layout(width='700px'))

rot_atom_pairs = mol.GetSubstructMatches(RotatableBondSmarts)
rot_bonds = list(set([mol.GetBondBetweenAtoms(*ap).GetIdx() for ap in rot_atom_pairs]))

Chem.rdDepictor.Compute2DCoords(mol)
Chem.Draw.rdMolDraw2D.PrepareAndDrawMolecule(d, mol, legend='Rotable in red', highlightBonds=rot_bonds)

d.WriteDrawingText("ligand_2D.png")
with open("ligand_2D.png", "rb") as f:
    data = f.read()
display_image = widgets.Image(value=data, format='png', width=700, height=500)

def recolor_bonds(fix_index):
    bond_colors = {}
    for bond in Chem.rdchem.Mol.GetBonds(mol):
        if bond.GetIdx() in fix_index:
            bond_colors[bond.GetIdx()] = (0.0,0.0,0.8)
        elif bond in rot_bonds:
            bond_colors[bond.GetIdx()] = (0.8,0.0,0.0)
    return bond_colors

def update_rotatable_bond(change):
    new_input = change.new
    if new_input.endswith(','):
        return
    if new_input == '':
        fix_bonds_idx = []
    else:
        fix_bonds_idx = [int(s) for s in new_input.split(',')]
    bond_colors = recolor_bonds(fix_bonds_idx)
    d.ClearDrawing()
    Chem.Draw.rdMolDraw2D.PrepareAndDrawMolecule(d, mol, legend='Rotable in red, Fixed in blue',
                                                 highlightBonds=rot_bonds, highlightBondColors=bond_colors)
    rot_bond_count = 0
    for bond in rot_bonds:
        if bond not in fix_bonds_idx:
            rot_bond_count += 1
    display_rotable.value = str(rot_bond_count)
    d.WriteDrawingText("ligand_2D.png")
    with open("ligand_2D.png", "rb") as f:
        data = f.read()
    display_image.value = data

select_fix.observe(update_rotatable_bond, names='value')
display(select_fix, display_rotable, display_amide, display_image)

Text(value='', description='Index of the bonds to fix:', layout=Layout(width='700px'), placeholder='Enter the …

Text(value='1', description='Number of rotable bonds :', layout=Layout(width='700px'), style=DescriptionStyle(…

Text(value='0', description='Number of amide bonds :', layout=Layout(width='700px'), style=DescriptionStyle(de…

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\xbc\x00\x00\x01\xf4\x08\x02\x00\x00\x00P;i\x88\x…

In [24]:
# @markdown ## Preparing ligand.pdbqt by prepare_ligand.py.
SMILES = Chem.rdmolfiles.MolToSmiles(mol)
rotable = ""

if select_fix.value == '':
    fix_bonds_idx = []
else:
    fix_bonds_idx = [int(s) for s in select_fix.value.split(',')]

for i in fix_bonds_idx:
    bond = mol.GetBondWithIdx(i)
    a1 = bond.GetBeginAtom().GetIdx()+1 # RDKit은 0부터 셈
    a2 = bond.GetEndAtom().GetIdx()+1
    rotable = rotable + ' -r "'+SMILES+'" -b '+str(a1)+" "+str(a2)

subprocess.run(f'mk_prepare_ligand.py -i ligand.sdf -o ligand.pdbqt{rotable}', shell=True)

if os.path.exists('ligand.pdbqt'):
    print('ligand PDBQT file has been generated.')
else:
    print('Failed to generate ligand PDBQT file.')

ligand PDBQT file has been generated.


## Prepare Receptor

In [ ]:
# @markdown ### Prepare Receptor.pdbq
# @markdown - Get used to the NGLViewer below!<br/><br/>
# @markdown - Left drag - rotate<br/>Right drag - translate<br/>Wheel scroll - zoom in/out <br/>Left long click - recenter
view2 = nv.NGLWidget()
view2._set_size('1000px','750px')

receptorfile = files.upload()
receptorfile_name = next(iter(receptorfile))
fixer = PDBFixer(filename=next(iter(receptorfile)))

with open('receptor-unprocessed.pdb', 'w') as f:
    PDBFile.writeFile(fixer.topology, fixer.positions, f)

view2.add_structure(nv.FileStructure('receptor-unprocessed.pdb'))
view2.add_spacefill("ion")
display(view2)

Saving 8S7V_MCR_NoUAA.pdb to 8S7V_MCR_NoUAA.pdb


NGLWidget()

### Chains to preserve

In [ ]:
amino_acids = ['ALA', 'CYS', 'ASP', 'GLU', 'PHE', 'GLY', 'HIS', 'ILE', 'LYS', 'LEU',
               'MET', 'ASN', 'PRO', 'GLN', 'ARG', 'SER', 'THR', 'VAL', 'TRP', 'TYR']
nucleic_acids = ['A', 'T', 'G', 'C', 'U', 'I', 'DA', 'DT', 'DG', 'DC', 'DI']

#@markdown In the above widget, chains are shown in different colors.
#@markdown Enter the chains you want to preserve.
chains_to_keep = 'ACD' # @param {type:"string"}
#@markdown To maintain metal ions in your system, enter the metal's residue name and its charge separated by colon (:).
#@markdown For multiple metal ions, separate them with commas (,).
metal_to_keep = '' # @param {type:"string"}
# @markdown The fixed receptor structure will be saved to 'receptor-fixed.pdb'.

metals_to_keep = {}

if metal_to_keep != '':
    metal_residues = metal_to_keep.split(',')

    for metal in metal_residues:
        metal_info = metal.split(':')
        print(f"adding metal: {metal_info[0]}, {metal_info[1]}")
        metals_to_keep[metal_info[0]] = metal_info[1]

print("Fixing the pdb file...", end='')
chains_to_delete = []

for chain in fixer.topology.chains():
    if chain.id not in chains_to_keep:
        print(f'removing: {chain.id}')
        chains_to_delete.append(chain.index)

#chains_to_delete = [] ##remove
fixer.removeChains(chains_to_delete)
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.findMissingResidues()
terminal_missing_res = []
chains = list(fixer.topology.chains())
for key in fixer.missingResidues.keys():
    if key[1] == 0 or key[1] == len(list(chains[key[0]].residues())):
        terminal_missing_res.append(key)
for key in terminal_missing_res:
    del fixer.missingResidues[key]
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(pH)

model = Modeller(fixer.topology, fixer.positions)
model.deleteWater()

hetero_res = []
for res in model.topology.residues():
    if res.name not in (amino_acids+nucleic_acids+list(metals_to_keep.keys())):
        hetero_res.append(res)
model.delete(hetero_res)

n_metals = 0
for res in model.topology.residues():
    if res.name in metals_to_keep:
        n_metals += 1

print("done.")

with open('receptor-fixed.pdb', 'w') as f:
    PDBFile.writeFile(model.topology, model.positions, f)

u = mda.Universe('receptor-fixed.pdb')
com = u.select_atoms('all').center_of_geometry()
u.atoms.translate(-com)

for chain in u.segments:
    first_residue = chain.residues[0]
    n_terminal_h = first_residue.atoms.select_atoms('name H')
    if len(n_terminal_h) > 0:
        n_terminal_h[0].name = 'H1'

if n_metals > 0:
    u.add_TopologyAttr('charge')
    for res in u.residues:
        if res.resname in metals_to_keep:
            res.atoms[0].charge =  float(metals_to_keep[res.resname])
    u.select_atoms(f'resname {" ".join(list(metals_to_keep.keys()))}').write('metals.pdbqt')

u.select_atoms('protein or nucleic').write('receptor-centered.pdb')

view3 = nv.NGLWidget()
view3._set_size('1000px','750px')
view3.add_structure(nv.FileStructure('receptor-centered.pdb'))
view3.add_licorice()
if n_metals > 0:
    view3.add_structure(nv.FileStructure('metals.pdbqt'))
    view3.add_spacefill("ion")
#view3.add_surface(lowResolution= True, smooth=1,opacity=0.5)
display(view3)

Fixing the pdb file...removing: B
removing: E
removing: E
removing: F
removing: F
removing: F
removing: G
removing: H
removing: H
removing: I
removing: I
removing: J
removing: K
removing: K
removing: L
done.


NGLWidget()

### In the above widget, select an atom to set it as a center for the grid box, OR...<br/>

In [ ]:
# @markdown Choose an atom to center manually.<br/>

center_atom = " " #@param {type:"string"}

f = open("receptor-centered.pdb", "r")
lines = f.readlines()
f.close()

# open metals.pdbqt if it exists and read lines
if n_metals > 0:
    f = open("metals.pdbqt", "r")
    lines_metals = f.readlines()
    f.close()
lines = lines + lines_metals

for line in lines:
  if center_atom.lower() in line.lower():
    center_x_user = float(line[30:38])
    center_y_user = float(line[38:46])
    center_z_user = float(line[46:54])
    print(f"The coordinate for the box center is x:{center_x_user:.1f} A, y:{center_y_user:.1f} A, z:{center_z_user:.1f} A.")
    break
else:
    print("The atom for the box center has not been selected.")

The coordinate for the box center is x:2.7 A, y:-5.0 A, z:-4.1 A.


In [ ]:
# @markdown When an atom is selected, its info will be shown in white at the upper left corner.<br/>
# @markdown Or you can enter the coordinates of the box center manually below.

center_x = "0.53" #@param {type:"string"}
center_y = "-0.044" #@param {type:"string"}
center_z = "-26.454" #@param {type:"string"}

# @markdown If you are not aware of the putative binding site, check the box below and leave the above boxes empty.

blind_docking = False #@param {type:"boolean"}

if center_x == " " and center_y == " " and center_z == " ":
    center_x = center_x_user
    center_y = center_y_user
    center_z = center_z_user

elif center_x != "" and center_y != "" and center_z != "":
    try:
        center_x = float(center_x)
        center_y = float(center_y)
        center_z = float(center_z)
    except:
        print("Box center coordinates are not read.")

elif blind_docking:
    center_x = 0.0
    center_y = 0.0
    center_z = 0.0

elif view3.picked != {}:
    center_x = view3.picked["atom1"]["x"]
    center_y = view3.picked["atom1"]["y"]
    center_z = view3.picked["atom1"]["z"]

else:
    print("The atom for the box center has not been selected.")

print(f"The coordinate for the box center is x:{center_x:.1f} A, y:{center_y:.1f} A, z:{center_z:.1f} A.")


The coordinate for the box center is x:0.5 A, y:-0.0 A, z:-26.5 A.


### Adjust Box

In [ ]:
# @markdown Adjust the box position/dimensions. <br/>
# @markdown The x-axis is shown in red, y-axis in green, and z-axis in blue. <br/>
# @markdown The default box size is calculated from the ligand size.

def find_ligand_dim():
    ligand = Chem.rdmolfiles.SDMolSupplier("ligand.sdf")[0]
    xs = []
    ys = []
    zs = []

    for coord in ligand.GetConformer().GetPositions():
        xs.append(coord[0])
        ys.append(coord[1])
        zs.append(coord[2])

    dx = max(xs)-min(xs)
    dy = max(ys)-min(ys)
    dz = max(zs)-min(zs)

    return int(sqrt(dx*dx+dy*dx+dz*dz))


view4 = nv.NGLWidget()
view4._set_size('1000px','750px')
view4.add_structure(nv.FileStructure('receptor-centered.pdb'))
if n_metals > 0:
    view4.add_structure(nv.FileStructure('metals.pdbqt'))
    view4.add_spacefill("ion")
view4.add_licorice()
# view4.add_surface(lowResolution= True, smooth=1,opacity=0.4)

x_select = widgets.BoundedFloatText(value=center_x, min=-1000, max=1000, step=0.5, description="center_x (A): ",
                                    continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='1000px'))

y_select = widgets.BoundedFloatText(value=center_y, min=-1000, max=1000, step=0.5, description="center_y (A): ",
                                    continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='1000px'))

z_select = widgets.BoundedFloatText(value=center_z, min=-1000, max=1000, step=0.5, description="center_z (A): ",
                                    continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='1000px'))

r = find_ligand_dim()

l_select = widgets.BoundedFloatText(value=r*3, min=r, max=1000, step=0.5, description="length_x (A) : ",
                                    continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='1000px'))

w_select = widgets.BoundedFloatText(value=r*3, min=r, max=1000, step=0.5, description="length_y (A) : ",
                                    continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='1000px'))

h_select = widgets.BoundedFloatText(value=r*3, min=r, max=1000, step=0.5, description="length_z (A) : ",
                                    continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='1000px'))


def draw_box(v, x, y, z, l, w, h, runtime):
    if runtime > 0:
        v._execute_js_code(f"""
        this.stage.removeComponent(this.stage.getComponentsByName("vertice1").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice2").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice3").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice4").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice5").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice6").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice7").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice8").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice9").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice10").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice11").first);
        this.stage.removeComponent(this.stage.getComponentsByName("vertice12").first)
        """)
    corner1 = [x+l/2, y+w/2, z+h/2]
    corner2 = [x-l/2, y+w/2, z+h/2]
    corner3 = [x-l/2, y-w/2, z+h/2]
    corner4 = [x+l/2, y-w/2, z+h/2]
    corner5 = [x+l/2, y+w/2, z-h/2]
    corner6 = [x-l/2, y+w/2, z-h/2]
    corner7 = [x-l/2, y-w/2, z-h/2]
    corner8 = [x+l/2, y-w/2, z-h/2]
    v._execute_js_code(
        f"""
        this.addShape("vertice1", [["cylinder", {corner1}, {corner2}, [1,1,1], [0.1]]]);
        this.addShape("vertice2", [["cylinder", {corner2}, {corner3}, [1,1,1], [0.1]]]);
        this.addShape("vertice3", [["cylinder", {corner3}, {corner4}, [1,1,1], [0.1]]]);
        this.addShape("vertice4", [["cylinder", {corner4}, {corner1}, [1,1,1], [0.1]]]);
        this.addShape("vertice5", [["cylinder", {corner1}, {corner5}, [1,1,1], [0.1]]]);
        this.addShape("vertice6", [["cylinder", {corner2}, {corner6}, [1,1,1], [0.1]]]);
        this.addShape("vertice7", [["cylinder", {corner3}, {corner7}, [0,0,1], [0.2]]]);
        this.addShape("vertice8", [["cylinder", {corner4}, {corner8}, [1,1,1], [0.1]]]);
        this.addShape("vertice9", [["cylinder", {corner5}, {corner6}, [1,1,1], [0.1]]]);
        this.addShape("vertice10", [["cylinder", {corner6}, {corner7}, [0,1,0], [0.2]]]);
        this.addShape("vertice11", [["cylinder", {corner7}, {corner8}, [1,0,0], [0.2]]]);
        this.addShape("vertice12", [["cylinder", {corner8}, {corner5}, [1,1,1], [0.1]]])
        """)

def update_x(change):
    x = change.new
    y = y_select.value
    z = z_select.value
    l = l_select.value
    w = w_select.value
    h = h_select.value
    draw_box(view4, x, y, z, l, w, h, 1)

def update_y(change):
    x = x_select.value
    y = change.new
    z = z_select.value
    l = l_select.value
    w = w_select.value
    h = h_select.value
    draw_box(view4, x, y, z, l, w, h, 1)

def update_z(change):
    x = x_select.value
    y = y_select.value
    z = change.new
    l = l_select.value
    w = w_select.value
    h = h_select.value
    draw_box(view4, x, y, z, l, w, h, 1)

def update_l(change):
    x = x_select.value
    y = y_select.value
    z = z_select.value
    l = change.new
    w = w_select.value
    h = h_select.value
    draw_box(view4, x, y, z, l, w, h, 1)

def update_w(change):
    x = x_select.value
    y = y_select.value
    z = z_select.value
    l = l_select.value
    w = change.new
    h = h_select.value
    draw_box(view4, x, y, z, l, w, h, 1)

def update_h(change):
    x = x_select.value
    y = y_select.value
    z = z_select.value
    l = l_select.value
    w = w_select.value
    h = change.new
    draw_box(view4, x, y, z, l, w, h, 1)

draw_box(view4, x_select.value, y_select.value, z_select.value, l_select.value, w_select.value, h_select.value, 0)
x_select.observe(update_x, names='value')
y_select.observe(update_y, names='value')
z_select.observe(update_z, names='value')
l_select.observe(update_l, names='value')
w_select.observe(update_w, names='value')
h_select.observe(update_h, names='value')

center_x = x_select.value
center_y = y_select.value
center_z = z_select.value
len_x = l_select.value
len_y = w_select.value
len_z = h_select.value

display(x_select, y_select, z_select, l_select, w_select, h_select, view4)

BoundedFloatText(value=-2.0379999999999825, description='center_x (A): ', layout=Layout(width='1000px'), max=1…

BoundedFloatText(value=-11.448000000000008, description='center_y (A): ', layout=Layout(width='1000px'), max=1…

BoundedFloatText(value=-3.1990000000000123, description='center_z (A): ', layout=Layout(width='1000px'), max=1…

BoundedFloatText(value=21.0, description='length_x (A) : ', layout=Layout(width='1000px'), max=1000.0, min=7.0…

BoundedFloatText(value=21.0, description='length_y (A) : ', layout=Layout(width='1000px'), max=1000.0, min=7.0…

BoundedFloatText(value=21.0, description='length_z (A) : ', layout=Layout(width='1000px'), max=1000.0, min=7.0…

NGLWidget()

### Bypass for flexible residues
- this doesn;t work right now, so leave the box blank and run the cell

In [ ]:
# @markdown Enter the flexible residues as
# @markdown "Chain ID:Residue number"<br/>
# @markdown For more than one flexible residues, separate them with commas without blanks. <br/>
# @markdown If there is none, leave it empty. <br/>
# @markdown **Please do not include HIS in flexible residues!**

flexible_residues = "" # @param {type:"string"}

flexFlag = False
flex_res = ""
selection = ""

if flexible_residues != "":
    try:
        for i, fr in enumerate(flexible_residues.split(',')):
            resinfo = fr.split(':')
            for res in u.residues:
                if res.segid == resinfo[0] and res.resnum == int(resinfo[1]):
                    flex_res = flex_res + f' -f {resinfo[0]}:{res.resname}:{resinfo[1]}'
                    selection = selection + f'(:{resinfo[0]} and {resinfo[1]})'
                    if i != (len(fr)-1):
                        selection = selection + " or "
            flexFlag = True
    except:
        print("Failed to recognize the flexible residue info.")

view5 = nv.NGLWidget()
view5._set_size('1000px','750px')
view5.add_structure(nv.FileStructure('receptor-centered.pdb'))
# if n_metals > 0:
    # view5.add_structure(nv.FileStructure('metals.pdbqt'))
    # view5.add_spacefill("ion")
view5.add_surface(lowResolution= True, smooth=1,opacity=0.4)

center_x = x_select.value
center_y = y_select.value
center_z = z_select.value
len_x = l_select.value
len_y = w_select.value
len_z = h_select.value

draw_box(view5, center_x, center_y, center_z, len_x, len_y, len_z, 0)

if flexFlag:
    view5.add_licorice(selection=selection, color='red')
    view5.add_surface(selection=selection, color='red',
                      lowResolution=True, smooth=1,opacity=0.8)
    display(view5)

else:
    print("No flexible residues set.")

No flexible residues set.


In [ ]:
# @markdown ### Prepare receptor.pdbqt.
# @markdown - this will set partial charges and create the PDBQT file
subprocess.run('obabel receptor-centered.pdb -O receptor-charged.pdbqt -rx --partialcharge gasteiger', shell=True)
subprocess.run(f'mk_prepare_receptor.py --write_pdbqt receptor-charged.pdbqt -o receptor.pdbqt --box_size {len_x} {len_y} {len_z} --box_center {center_x} {center_y} {center_z} --flexres {flex_res} --skip_gpf', shell=True)

# if flexFlag == False:
#   print("false")
#   subprocess.run(f'mk_prepare_receptor.py --read_pdb receptor-charged.pdbqt -p receptor.pdbqt --box_size {len_x} {len_y} {len_z} --box_center {center_x} {center_y} {center_z}' , shell=True) # --flexres {flex_res}', shell=True)
# elif flexFlag == True:
#   print("true")
#   subprocess.run(f'mk_prepare_receptor.py --read_pdb receptor-charged.pdbqt -p receptor_flex.pdbqt --box_size {len_x} {len_y} {len_z} --box_center {center_x} {center_y} {center_z} --flexres {flex_res}', shell=True)

f = open('receptor-charged.pdbqt', 'r')
lines = f.readlines()
f.close()
new_lines = []
remove_flags = ['tors', 'between', 'root', 'branch', 'status']
for line in lines:
  if not any(flag in line.lower() for flag in remove_flags):
    new_lines.append(line)

f = open('receptor.pdbqt', 'w')
f.writelines(new_lines)
f.close()

if flexFlag == True:
    target = 'receptor_flex.pdbqt'
elif flexFlag == False:
    target = 'receptor.pdbqt'

if os.path.exists(f'{target}'):
    print('receptor PDBQT file generated.')
else:
    print('Failed to generate receptor PDBQT file.')

if n_metals > 0:
    subprocess.run(f'tail -n {n_metals} metals.pdbqt >> {target}', shell=True)

receptor PDBQT file generated.


## If you already have a PDBQT file, begin here!
- you will need to set centers manually

### Bypass for protein prep

In [5]:
# @markdown When an atom is selected, its info will be shown in white at the upper left corner.<br/>
# @markdown Or you can enter the coordinates of the box center manually below.

flexFlag = False
blind_docking = False

center_x = "0.53" #@param {type:"string"}
center_y = "-0.044" #@param {type:"string"}
center_z = "-26.454" #@param {type:"string"}

len_x = "28" #@param {type:"string"}
len_y = "28" #@param {type:"string"}
len_z = "28" #@param {type:"string"}

#Ligand_number = "3" #@param {type:"string"}


## Docking with Autodock Vina
-If dies with a particular ligand, check for CG and CG0 astom types and change to C in pdbqt file. can test by running qvina command at the command line

In [11]:
# @markdown - set number of repetitions
number_reps = 1 #@param {type:"number", min:1, max:10, step:1}


In [25]:
best_scores = []
for run_number in range(1,number_reps+1,1):
    # @markdown ### Dock with QuickVina
    output = f"ligand{Ligand_number}_out_{run_number}.pdbqt"
    # @markdown Set the exhaustiveness value. For blind dockings, increase it to larger values.
    exhaustiveness = 32 #@param {type:"slider", min:8, max:512, step:8}
    # @markdown The docking log below will also be saved at docking_log.txt.

    if flexFlag:
        rigid = "receptor = receptor.pdbqt\n"
        flex_rec = "flex = receptor_flex.pdbqt\n"
    else:
        rigid = "receptor = receptor.pdbqt\n"
        flex_rec = ""

    with open('config', 'w') as f:
        f.write(rigid)
        f.write("ligand = ligand.pdbqt\n")
        f.write("center_x = "+str(center_x)+'\n')
        f.write("center_y = "+str(center_y)+'\n')
        f.write("center_z = "+str(center_z)+'\n')
        f.write("size_x = "+str(len_x)+'\n')
        f.write("size_y = "+str(len_y)+'\n')
        f.write("size_z = "+str(len_z)+'\n')
        f.write("num_modes = 10\n")
        f.write("out = "+output+'\n')
        f.write(f"log = docking_log{Ligand_number}_{run_number}.txt\n")
        f.write("exhaustiveness = "+str(exhaustiveness)+'\n')
        f.write(flex_rec)

    if blind_docking:
        p = subprocess.Popen('/content/qvina/bin/qvina-w --config config', shell=True, bufsize=0, stdout=subprocess.PIPE, encoding='utf-8')
    else:
        p = subprocess.Popen('/content/qvina/bin/qvina2.1 --config config', shell=True, bufsize=0, stdout=subprocess.PIPE, encoding='utf-8')

    while p.poll() == None:
        out = p.stdout.read(1)
        print(out, end='')

    docking_file = f"docking_log{Ligand_number}_{run_number}.txt"
    df = open(docking_file, 'r')
    lines = df.readlines()
    df.close()
    for line in lines:
      parts = line.split()
      if len(parts) >1:
        if parts[0] == "1":
          best_scores.append(parts[1])
    files.download(docking_file)
    files.download(output)

for score in best_scores:
    print(score)


############################################################################
# If you used Quick Vina 2 in your work, please cite:                      #
#                                                                          #
# Amr Alhossary, Stephanus Daniel Handoko, Yuguang Mu, and Chee-Keong Kwoh,#
# Fast, Accurate, and Reliable Molecular Docking with QuickVina 2,         #
# Bioinformatics (2015), doi: 10.1093/bioinformatics/btv082                #
#                                                                          #
# You are also encouraged to cite Quick Vina 1:                            #
# Stephanus Daniel Handoko, Xuchang Ouyang, Chinh Tran To Su, Chee Keong   #
# Kwoh, Yew Soon Ong,                                                      #
# QuickVina: Accelerating AutoDock Vina Using Gradient-Based Heuristics for#
# Global Optimization,                                                     #
# IEEE/ACM Transactions on Computational Biology and Bioinformatics,vol.9, #

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

-5.8


## Shifts - extra code to find the shifts between the unprocessed and centered PDB files

### Find a shift vector

In [ ]:
f = open('receptor-centered.pdb', 'r')
lines_centered = f.readlines()
f.close()

f = open('receptor-unprocessed.pdb', 'r')
lines_unprocessed = f.readlines()
f.close()

for line in lines_unprocessed:
  if line.startswith('ATOM'):
    parts = line.split()
    print(line)
    x_orig = float(parts[6])
    y_orig = float(parts[7])
    z_orig = float(parts[8])
    break

for line in lines_centered:
  if line.startswith('ATOM'):
    parts = line.split()
    print(line)
    x_centered = float(parts[6])
    y_centered = float(parts[7])
    z_centered = float(parts[8])
    break

shift_vector = [x_centered-x_orig, y_centered-y_orig, z_centered-z_orig]
print(shift_vector)

ATOM      1  N   ALA A   1     178.503 185.688 100.052  1.00  0.00           N  

ATOM      1  N   ALA A   1       1.626  13.308 -38.973  1.00  0.00      A    N  

[-176.87699999999998, -172.38, -139.025]


### Shift a single set of coordinates

In [ ]:
# shift a set of coordinates

import_center = [174.839, 160.932, 135.826]
new_center = []
for center, shift in zip(import_center, shift_vector):
    new_center.append(center+shift)
print(new_center)

[-2.0379999999999825, -11.448000000000008, -3.1990000000000123]


### Find a ligand in the PDB and shift it

In [ ]:
# find and shift a bound ligand

f = open('receptor-unprocessed.pdb', 'r')
lines_unprocessed = f.readlines()
f.close()

just_ligand = []
current_num = []
for line in lines_unprocessed:
  if 'LDP' in line and 'ter' not in line.lower():
    parts = line.split()
    current_num.append(int(parts[1]))
    if (len(current_num) > 1 and (current_num[-1] != (int(current_num[-2]) +1))) :
      break
    x = float(parts[6])
    y = float(parts[7])
    z = float(parts[8])
    new_x = x+shift_vector[0]
    new_y = y+shift_vector[1]
    new_z = z+shift_vector[2]
    current_num.append(parts[1])
    header = f'{parts[0]} {parts[1]}  {parts[2]:<4} {parts[3]} {parts[4]} {parts[5]}'
    coords = f'{new_x:7.3f} {new_y:7.3f} {new_z:7.3f}'
    footer = '1.00  0.00'
    print(f"{header}     {coords}  {footer}          {parts[-1]:>2} ")
    just_ligand.append(f"{header}     {coords}  {footer}          {parts[-1]:>2} \n")

f = open('just_ligand.pdb', 'w')
f.writelines(just_ligand)
f.close()

HETATM 4722  C7   LDP C 2     -10.688   2.225  -1.730  1.00  0.00           C 
HETATM 4723  C1   LDP C 2      -9.224   1.965  -2.322  1.00  0.00           C 
HETATM 4724  C4   LDP C 2      -6.570   1.535  -3.529  1.00  0.00           C 
HETATM 4725  C2   LDP C 2      -8.113   1.423  -1.478  1.00  0.00           C 
HETATM 4726  C6   LDP C 2      -8.940   2.270  -3.770  1.00  0.00           C 
HETATM 4727  C5   LDP C 2      -7.653   2.057  -4.345  1.00  0.00           C 
HETATM 4728  C3   LDP C 2      -6.785   1.206  -2.074  1.00  0.00           C 
HETATM 4729  O1   LDP C 2      -5.726   0.711  -1.347  1.00  0.00           O 
HETATM 4730  O2   LDP C 2      -5.347   1.352  -4.136  1.00  0.00           O 
HETATM 4731  C8   LDP C 2     -11.898   1.899  -2.770  1.00  0.00           C 
HETATM 4732  N1   LDP C 2     -11.972   2.890  -3.924  1.00  0.00           N 


## Extra code to clean up a PDBQT file

In [ ]:
f = open(output, 'r')
lines = f.readlines()
f.close()

new_lines = []
remove_flags = ['tors', 'between', 'root', 'branch', 'status']
for line in lines:
  if not any(flag in line.lower() for flag in remove_flags):
    new_lines.append(line)

f = open('ligand_clean.pdbqt', 'w')
f.writelines(new_lines)
f.close()